#### 1. 데이터 준비
프랑스어로 번역된 영어 문장 쌍을 사용합니다. 데이터셋은 Kaggle의 English to French dataset을 사용할 수 있습니다.


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# 간단한 데이터셋 예제
english_sentences = ["hello", "how are you", "what is your name"]
french_sentences = ["bonjour", "comment ça va", "quel est ton nom"]

# 토크나이저를 사용해 텍스트를 시퀀스로 변환
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)

fr_tokenizer = Tokenizer()
fr_tokenizer.fit_on_texts(french_sentences)
fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences)

# 시퀀스를 패딩
max_seq_len = max(max([len(seq) for seq in eng_sequences]), max([len(seq) for seq in fr_sequences]))
eng_padded = pad_sequences(eng_sequences, maxlen=max_seq_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_seq_len, padding='post')

# 단어 수 계산
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1


#### 2. Seq2Seq 모델 정의
Seq2Seq 모델의 기본 구성인 인코더와 디코더를 정의합니다.

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# 인코더
encoder_inputs = Input(shape=(max_seq_len,))
encoder_embedding = Embedding(eng_vocab_size, 64)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(64, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# 디코더
decoder_inputs = Input(shape=(max_seq_len,))
decoder_embedding = Embedding(fr_vocab_size, 64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 4, 64)     │        576 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 4, 64)     │        576 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 64),      │     33,024 │ embedding_4[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 4, 64),   │     33,024 │ embedding_5[0][0… │
│                     │ (None, 64),       │            │ lstm_4[0][1],     │
│                     │ (None, 64)]       │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 4, 9)      │        585 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 67,785 (264.79 KB)

 Trainable params: 67,785 (264.79 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 인코더 모델 정의
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 모델 정의
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding2 = Embedding(fr_vocab_size, 64)(decoder_inputs)
decoder_lstm2 = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs2, state_h2, state_c2 = decoder_lstm2(
    decoder_embedding2, initial_state=decoder_states_inputs
)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [13]:
def decode_sequence(input_seq):
    # 인코더 예측
    states_value = encoder_model.predict(input_seq)

    # 시작 토큰으로 디코더 초기화
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_tokenizer.word_index['start']

    # 반복을 통해 시퀀스를 생성
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측된 단어를 텍스트로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # 종료 조건 확인
        if sampled_word == 'end' or len(decoded_sentence) > max_seq_len:
            stop_condition = True

        # 디코더의 다음 입력으로 업데이트
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence


#### 3. 모델 학습
모델을 학습시킵니다. 디코더의 출력이 시퀀스 전체이기 때문에 sparse_categorical_crossentropy 손실함수를 사용합니다.

In [8]:
# 디코더 출력 데이터 준비
fr_padded_output = np.expand_dims(fr_padded, -1)

# 모델 학습
model.fit([eng_padded, fr_padded], fr_padded_output, epochs=100, batch_size=64)


Epoch 1/100


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 2.1997
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.1904
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.1810
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.1716
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.1619
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2.1518
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.1411
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.1298
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.1177
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.1047
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.0905
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.0749
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 2.0578
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 2.0389
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 2.0180
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━━

#### 4. 모델 평가 및 번역 시도
학습된 모델을 사용하여 영어 문장을 프랑스어로 번역합니다.

In [15]:
def decode_sequence(input_seq):
    # 인코더 예측
    states_value = encoder_model.predict(input_seq)

    # 시작 토큰으로 디코더 초기화
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fr_tokenizer.word_index['start']

    # 반복을 통해 시퀀스를 생성
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측된 단어를 텍스트로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # 종료 조건 확인
        if sampled_word == 'end' or len(decoded_sentence) > max_seq_len:
            stop_condition = True

        # 디코더의 다음 입력으로 업데이트
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence

# 테스트
for seq_index in range(3):
    input_seq = eng_padded[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(f"Input sentence: {english_sentences[seq_index]}")
    print(f"Decoded sentence: {decoded_sentence}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


KeyError: 'start'